In [82]:
import pandas as pd
import numpy as np
import itertools

In [85]:
def multiply_by_x(n, x=100):
    """
    Multiplies n by 100 (or x): (e.g. n = 5, returns [5,5,5...(100), 5]

    Parameters
    ----------
    n : int

    Returns
    -------
    list 100 n's
    """

    return [n] * x


def get_scale(wiggle, scale_to=100):
    """

    Parameters
    ----------
    wiggle : pandas.Series
        Series of values of any length
    Returns
    -------
        Series of values that is scale (length is always 100).
    """

    # Need to adjust series such that it has at least 100 parts.
    # Required since stepper will iterate from 0.01..1 and x will
    # iterate from 0..99.
    if len(wiggle) == scale_to:  # no need to do any calculating.
        return wiggle
    elif len(wiggle) == 1:  # return 100/n of these values
        return pd.Series(
            list(
                itertools.chain.from_iterable(
                    [multiply_by_x(w, x=scale_to) for w in wiggle]
                )
            )
        )
    elif len(wiggle) < scale_to:  # multiply everything by scaling factor, this guarantees it is divisible by scaling factor
        wiggle = pd.Series(
            list(
                itertools.chain.from_iterable(
                    [multiply_by_x(w, x=scale_to) for w in wiggle]
                )
            )
        )

    dist = [0] * scale_to  # final series length
    x = 0  # iterate through dist list
    step = 1/float(scale_to)  # stepper, increments increase by this number
    y = 0  # number of values in each stepwise bin

    # iterate through each value until it reaches next step, then averages
    # (step = 1% -> 2%, or 2% -> 3%, etc. if we are trying to scale to 100%)
    for pos, value in enumerate(wiggle):
        if (float(pos + 1) / len(wiggle)) < step:  # if we haven't reached the next step, add value to bin (dist[x])
            # print("{} < {}, dist[{}] = {}".format((float(pos + 1) / len(wiggle)), step, x, value))
            y = y + 1
            dist[x] = dist[x] + value
        elif (float(pos + 1) / len(wiggle) == 1):  # if we have reached the last step, break loop
            y = y + 1
            dist[x] = dist[x] + value
            break
        else:  # if we have passed the next step, divide total value in dist[x] by y (number of values) to get average of bin, then move on (iterate x)
            dist[x] = dist[x] / float(y)
            step = step + 1/float(scale_to)
            x = x + 1
            dist[x] = value
            y = 1
    
    try:
        dist[x] = dist[x] / float(y)
    except ZeroDivisionError as e:
        print("Got zero series, won't scale.", e, wiggle)
    except IndexError as e:
        pass
    return pd.Series(dist)

In [88]:
wiggle = pd.Series(range(100))
get_scale(wiggle, 50)

0      0.0
1      1.5
2      3.5
3      5.5
4      7.5
5     10.0
6     12.0
7     13.5
8     15.5
9     17.5
10    19.5
11    21.5
12    23.5
13    25.5
14    27.5
15    29.5
16    31.5
17    34.0
18    36.5
19    38.5
20    40.5
21    42.5
22    44.5
23    46.5
24    48.5
25    50.5
26    52.5
27    54.5
28    56.5
29    58.5
30    60.5
31    62.5
32    64.5
33    66.5
34    68.5
35    70.5
36    72.5
37    74.5
38    76.5
39    78.5
40    80.5
41    82.5
42    84.5
43    86.5
44    88.5
45    90.5
46    92.5
47    94.5
48    96.5
49    98.5
dtype: float64

In [50]:
7/2

3

In [30]:
97/2.0

48.5